In [1]:
import urllib3
urllib3.disable_warnings()
http = urllib3.PoolManager()
url = "https://datausa.io/about/attributes/geo/state/1/"
response = http.request('GET', url)

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.data, 'html.parser')

In [3]:
table=soup.find('table', class_='geo')

In [4]:
IDs=[]
Names=[]
for row in table.findAll("tr"):
    cells = row.findAll("td")
    if len(cells)==2:
        IDs.append(cells[0].find(text=True))
        Names.append(cells[1].find(text=True))

In [5]:
from pyhive import hive
conn = hive.Connection(host='sandbox.hortonworks.com', port=10000)
cursor = conn.cursor()

In [32]:
cursor.execute('use kcbda_raw')
cursor.execute('CREATE EXTERNAL TABLE DATAUSA_STATE (ID_STATE string, DE_STATE string) ROW FORMAT DELIMITED FIELDS TERMINATED BY "," STORED AS TEXTFILE LOCATION "/kcbda/RAW/DATAUSA_STATE"')

In [37]:
query = "INSERT INTO TABLE DATAUSA_STATE VALUES "
for id, name in list(zip(IDs, Names)):
    query += "('" + str(id) + "', '" + str(name) + "'),"
query = query[:-1]
cursor.execute(query)

In [47]:
import hdfs
from hdfs import InsecureClient
client = InsecureClient('http://sandbox.hortonworks.com:50070', user='hdfs', timeout=1000)

In [ ]:
url = 'http://api.datausa.io/api/csv/?show=geo&sumlevel=state&required=avg_wage&year=latest'
to_path ='/kcbda/RAW/DATAUSA_AVG/2015.csv'
response = http.request('GET', url)
client.write(to_path, response.data)

In [52]:
url = 'http://api.datausa.io/api/csv/?show=geo&sumlevel=state&required=high_school_graduation&year=latest'
to_path ='/kcbda/RAW/DATAUSA_HIGH_SCHOOL/2017.csv'
response = http.request('GET', url)
client.write(to_path, response.data)